<a href="https://colab.research.google.com/github/rk0803/Assignment3/blob/main/Assignment3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
torch.cuda.is_available()

True

In [6]:
def get_device():
  if torch.cuda.is_available():
    return torch.device('cuda')
  else:
    return torch.device('cpu')
device=get_device()
device

device(type='cuda')

In [7]:
def to_device(data,device):
  if isinstance(data,(list,tuple)):
      return[to_device(x,device) for x in data]
  return data.to(device,non_blocking=True)

In [14]:
from torch.utils.data import Dataset

In [15]:
class MySet(Dataset):
  def __init__(self,csv_file):
    self.data=pd.read_csv(csv_file)

  def __getitem__(self,index):
    r=self.data.iloc[index]
    label=torch.tensor(r.is_up_day,dtype=torch.long)
    sample=self.normalize(torch.tensor([r.open,r.high,r.low,r.close]))
    return sample, label

  def __len__(self):
    return len(self.data)

In [16]:
train_set=torchvision.datasets.MNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([transforms.ToTensor()])
)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [17]:
train_loader=torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

In [18]:
class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)
    def __len__(self):
        return len(self.dl)

In [19]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
   def __init__(self):
        super().__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 12, 5)
        self.fc1 = nn.Linear(202, 120) #202 = 12*4*4 + 10
        self.fc2 = nn.Linear(120, 84)
        self.out = nn.Linear(84, 28) #28 = 10 for image label and 18 for sum

   def forward(self, x,y):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,kernel_size=2,stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,kernel_size=2,stride=2)
        x = x.reshape(-1,12*4*4)
        x = torch.cat((x,y),dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=202, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (out): Linear(in_features=84, out_features=28, bias=True)
)


In [20]:
import torch.optim as optim

In [21]:
def get_num_correct(preds,labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

Create a set of 60000 random digits in batches of 100

In [33]:
import numpy as np
import random
from random import randrange
yr=np.array([[random.randrange(0,9,1) for j in range(100)] for i in range(600)])
yr=torch.as_tensor(yr)
yr=to_device(yr,device)


In [34]:
#training the Neural Net created with all batches
net=Net()
to_device(net,device)
trainloader=torch.utils.data.DataLoader(train_set,batch_size=100)
optimizer=optim.SGD(net.parameters(),lr=0.01,momentum=0.9)
for epoch in range(50):
  total_loss=0
  total_correct=0
  i=0
  for batch in train_loader:
    batch=to_device(batch,device)
    images,labels=batch
    #images=to_device(images,device)
    y=yr[i] # pick up the batch of random digits
    i+=1
    y1=to_device(F.one_hot(y,10),device) # convert it into one-vector
    sumlabint=y+labels   # Create the output labels for sum of image digit and random digit
    preds=net(images,y1)
    img1=preds[0:,:10]  #Extract the first output (image digit vector)
    imintsum=preds[0:,10:] # Extract the second output (sum of random and image digit)
    #Calculate the loss
    loss1=F.cross_entropy(img1,labels)
    loss2=F.cross_entropy(imintsum,sumlabint)
    loss=loss1+loss2
    optimizer.zero_grad()
    loss.backward() # Calculate the gradient
    optimizer.step() # Update the weights
    total_loss += loss.item()
    total_correct+=(get_num_correct(img1,labels))
  print("epoch ", epoch,"total_correct: ", total_correct, "loss ",total_loss)   



epoch  0 total_correct:  43633 loss  1986.0862352848053
epoch  1 total_correct:  57760 loss  1233.1535388231277
epoch  2 total_correct:  58418 loss  824.2203392386436
epoch  3 total_correct:  58703 loss  550.9549767673016
epoch  4 total_correct:  58852 loss  232.06129666417837
epoch  5 total_correct:  58991 loss  118.96101630851626
epoch  6 total_correct:  59127 loss  85.54578295350075
epoch  7 total_correct:  59281 loss  68.11687653698027
epoch  8 total_correct:  59345 loss  57.98299099504948
epoch  9 total_correct:  59434 loss  50.75924453092739
epoch  10 total_correct:  59478 loss  45.047040107194334
epoch  11 total_correct:  59508 loss  43.68901982344687
epoch  12 total_correct:  59546 loss  37.87449435936287
epoch  13 total_correct:  59572 loss  36.55079263006337
epoch  14 total_correct:  59612 loss  32.21810955717228
epoch  15 total_correct:  59645 loss  29.508404589490965
epoch  16 total_correct:  59649 loss  29.01106704468839
epoch  17 total_correct:  59707 loss  25.62930539983

To test the model on the test set, test is downloaded (if needed) and the model is tested on the test set.
Data is picked up in batches of hundred and number of correctly predicted labels is counted to evaluate the model. Although a Confusion Matrix is generally created to test the accuracy, but I dont know yet how to create it using pytorch functionalities(If any).
So I have calculated total number of correct labels predicted on MINST digits and sum.

In [35]:
test_set=torchvision.datasets.MNIST(
    root='./data'
    ,train=False
    ,download=True
    ,transform=transforms.Compose([transforms.ToTensor()])
)

In [37]:
yr=np.array([[random.randrange(0,9,1) for j in range(100)] for i in range(100)])
yr=torch.as_tensor(yr)
yr=to_device(yr,device)

In [41]:
testloader=torch.utils.data.DataLoader(test_set,batch_size=100)
total_correct_dig=0
total_correct_sum=0
i=0
for batch in testloader:
    batch=to_device(batch,device)
    images,labels=batch
    y=yr[i] # pick up the batch of random digits
    i+=1
    y1=to_device(F.one_hot(y,10),device) # convert it into one-vector
    sumlabint=y+labels   # Create the output labels for sum of image digit and random digit
    preds=net(images,y1)
    img1=preds[0:,:10]  #Extract the first output (image digit vector)
    imintsum=preds[0:,10:] # Extract the second output (sum of random and image digit)
    total_correct_dig+=(get_num_correct(img1,labels))
    total_correct_sum+=(get_num_correct(imintsum,sumlabint))
print("total_correct_digits: ", total_correct_dig, "total_correct_sum: ", total_correct_sum) 



total_correct_digits:  9917 total_correct_sum:  9897
